In [2]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import datetime

amap=pd.read_hdf('Processed_Data/amap.h5',key='1210update')
NEWday=pd.read_excel('./DataSource/DailyUpdateForTG.xlsx')
NEWday=pd.melt(NEWday,id_vars=['城市'],var_name='date', value_name='traffic')
updated=pd.concat([amap,NEWday]).drop_duplicates()
updated.to_hdf('Processed_Data/amap.h5',key='1210update')

amap=updated
amap['date']=pd.to_datetime(amap['date'])
amap['WeekDay']=amap['date'].dt.strftime('%A')


def getyr(d):
    return str(d.isocalendar()[0])
def getdt(d):
    return str(d.date())[-5:]
def getMon(d):
    return str(d.date()).split('-')[1]

amap['yr']=amap['date'].apply(getyr)
amap['dt']=amap['date'].apply(getdt)
amap['mon']=amap['date'].apply(getMon)

amap=amap.sort_values(by='date',ascending=True).reset_index(drop=True)

amap_2019=amap[amap['yr'].str.contains('2019')]
amap_2019=amap_2019.groupby(by=['城市','mon','WeekDay'])['traffic'].mean().reset_index() # 当的traffic 变化
amap_2019=amap_2019.rename(columns={"traffic":"traffic_19_same_wd_mean"})

amap_2022=amap[amap['yr']>='2022']
amap_2022=amap_2022.merge(amap_2019,how='left',on=['城市','mon','WeekDay'])

amap_2022['traffic_last7d_mean'] = amap_2022.groupby('城市')['traffic'].transform(lambda x: x.rolling(7, 7).mean())
amap_2022['traffic_change']=amap_2022['traffic_last7d_mean']/amap_2022.groupby('城市')['traffic_last7d_mean'].transform(lambda x: x.shift(periods=7))-1
amap_2022['traffic_status']=amap_2022['traffic']/amap_2022['traffic_19_same_wd_mean']
amap_2022['traffic_status_last7d_mean']=amap_2022.groupby('城市')['traffic_status'].transform(lambda x: x.rolling(7, 1).mean())

amap_2022.sort_values(by='date').tail(2)

,城市,date,traffic,WeekDay,yr,dt,mon,traffic_19_same_wd_mean,traffic_last7d_mean,traffic_change,traffic_status,traffic_status_last7d_mean
36535,哈尔滨,2023-01-03,1.54,Tuesday,2023,01-03,01,1.746,1.547143,0.042348,0.882016,0.851003
36599,肇庆,2023-01-03,1.36,Tuesday,2023,01-03,01,1.442,1.291429,0.008929,0.943135,0.898776


In [3]:
# 2.0 TG 流动性影响 by 人口拆分
city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
city_popu['常住人口']=pd.to_numeric(city_popu['常住人口'])

df_amap=amap_2022.merge(city_popu,on='城市',how='inner')

def catgorize2(g):
    g=g-1
    level='5 社会面流动性无影响'
    g=float(g)
    if g <=-0.25:
        level= '0 社会面基本停滞'
    elif -0.25<g<=-0.15:
        level ='1 社会面流动性显著降低'
    elif -0.15<g<=-0.05:
        level ='4 社会面流动性轻微降低'
    elif g>-0.05:
        level ='5 社会面流动性无影响'
    return level

df_amap['traffic_change_level']=df_amap['traffic_status_last7d_mean'].apply(catgorize2)

df_amap['date']=pd.to_datetime(df_amap['date'])

pv_value=pd.pivot_table(df_amap, values='常住人口', index=['date'],columns=["traffic_change_level"], aggfunc=np.sum)
pv_value=pv_value.div(pv_value.sum(axis=1), axis=0)
pv_value['sum']=1
pv_value=pv_value[['sum','0 社会面基本停滞', '1 社会面流动性显著降低', '4 社会面流动性轻微降低', '5 社会面流动性无影响']]
pv_value=pv_value.fillna(0)
pv_value.tail(22).to_clipboard(header=None)
pv_value.tail(3)

traffic_change_level,sum,0 社会面基本停滞,1 社会面流动性显著降低,4 社会面流动性轻微降低,5 社会面流动性无影响
date,,,,,
2023-01-01,1,0.003373,0.300379,0.551639,0.144609
2023-01-02,1,0.003373,0.297610,0.599259,0.099758
2023-01-03,1,0.003373,0.159039,0.650648,0.186940


In [4]:
# WoW vs. COVID
%load_ext autoreload
%autoreload 2
from colored_list import dafeng_city,jidui_city,riqi

df_covid=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')
dfo=amap_2022.merge(df_covid,on=['城市'],how='left')
dfo['date']=dfo['date'].astype('string') 
dfo=dfo[dfo['date']==riqi].sort_values(by='常住人口',ascending=False).head(70)

d=dfo[dfo['date']==riqi][['城市','date','traffic_status_last7d_mean']].reset_index(drop=True).rename(columns={'traffic_status_last7d_mean':'amap'})
d['date']=pd.to_datetime(d['date'])
d.to_hdf('Processed_Data/city_compare.h5',key='amap')

 
o=dfo.copy()
o['size']=np.nan
o['group']='其他'
c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='较早达峰'
o=o.sort_values(by='group',ascending=False)

show=o[['城市','progress_2','traffic_change','size','group']]
show.to_clipboard(index=False,header=None)

print(riqi)
show.head(3)

2023-01-03


,城市,progress_2,traffic_change,size,group
36537,重庆,0.0,0.092683,NaN,较早达峰
36532,徐州,1.0,0.055745,NaN,较早达峰
36581,西宁,5.0,0.065058,NaN,较早达峰


In [5]:
# as of 2019 vs. COVID

show=o[['城市','progress_2','traffic_status_last7d_mean','size','group']] #,'progress_2'
show.to_clipboard(index=False,header=None)

print(riqi)
show.head(3)


2023-01-03


,城市,progress_2,traffic_status_last7d_mean,size,group
36537,重庆,0.0,0.905668,NaN,较早达峰
36532,徐州,1.0,0.882109,NaN,较早达峰
36581,西宁,5.0,0.889350,NaN,较早达峰


In [6]:
# WoW vs. GDP

# %load_ext autoreload
# %autoreload 2
# from colored_list import dafeng_city,jidui_city,riqi

city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
dfo=amap_2022.merge(city_popu,on=['城市'],how='left')

dfo['date']=dfo['date'].astype('string') 
o=dfo[dfo['date']==riqi].sort_values(by='常住人口',ascending=False).head(70)

o=o[['城市','GDP','traffic_change']] #,'progress_2'
o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='较早达峰'

o=o.sort_values(by='group',ascending=False)
o.to_clipboard(index=False,header=None)
print(riqi)
o.head(2)

2023-01-03


,城市,GDP,traffic_change,size,group
36537,重庆,25003.0,0.092683,NaN,较早达峰
36510,济南,10141.0,0.144330,NaN,较早达峰


In [8]:
# as of 2019 vs. GDP
o=dfo[dfo['date']==riqi].sort_values(by='常住人口',ascending=False).head(70)

o=o[['城市','GDP','traffic_status_last7d_mean']] #,'progress_2'
o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='较早达峰'

o=o.sort_values(by='group',ascending=False)
o.to_clipboard(index=False,header=None)
print(riqi)
o.head(2)

2023-01-03


,城市,GDP,traffic_status_last7d_mean,size,group
36537,重庆,25003.0,0.905668,NaN,较早达峰
36510,济南,10141.0,0.890355,NaN,较早达峰
